In [1]:
import lib
import numpy as np
import os

# Hartree Fock computation of the ground state of the Helium atom

In this Jupyter Notebook we are going to compare the groundstate energies obtained with the analytical formula for the two body integrals with Montecarlo integration. In both cases, the Self-Consistent Field method is used.

### Basis functions
First we write the basis we are going to use

In [2]:
ALPHA_1 =  0.298073
ALPHA_2 =  1.242567
ALPHA_3 =  5.782948
ALPHA_4 = 38.474970
ALPHA = [ALPHA_1, ALPHA_2, ALPHA_3,ALPHA_4]

def He_wf(x, y, z, n):
    """
    Evaluates the n^th eigenfunction of the Hydrogen atom with exponential coefficient alpha_i. 
    The units of x are atomic units. 

    Parameters
    ==========
    x, y, z : float or np.ndarray
        Position in which to evaluate the wave function
    n : int
        Number of the wave function

    Returns
    =======
    float or np.ndarray
    """

    if n == 1:
        alpha = ALPHA_1
    elif n == 2:
        alpha = ALPHA_2
    elif n == 3:
        alpha = ALPHA_3
    elif n == 4:
        alpha = ALPHA_4
    else:
        alpha = None

    r2 = x**2 + y**2 + z**2

    return np.exp(-alpha*r2)

def He_wf_basis(R,k):
    """
    Evaluates the k^th eigenfunction of the Hydrogen atom with exponential coefficient alpha_i. 
    The units of x are atomic units. 

    Parameters
    ==========
    R : np.ndarray(3,N)
        Position in which to evaluate the wave function
    k : int
        Number of the wave function

    Returns
    =======
    float or np.ndarray(N)
    """
    return He_wf(R[0],R[1],R[2],k)

We now write the functions to calculate the integrands of the two-body integrals

In [3]:
def integrand_2(R):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    return 1/r12

def norm_product(p, r, q, s):
    a = ALPHA[p-1] + ALPHA[q-1]
    b = ALPHA[r-1] + ALPHA[s-1]
    PROD = (np.pi/a)**1.5 * (np.pi/b)**1.5
    
    return PROD

def He_two_body_integrand(p, r, q, s):
    """
    Integrand that goes into the monte carlo method in order to compute the 
    two electron integrals.

    Parameters
    ==========
    indices : int np.ndarray(4)
        Indices of the wave function

    Returns
    =======
    function
    """
    
    f = lambda R: integrand_2(R)*norm_product(p, r, q, s)
    
    return f

### One- and two- body integral computations
We first initialize some necessary parameters

In [4]:
N_electrons = 2
N_basis = 4
N_points = 1000000
integrals_file = "integrals_He.npy"

normalized_wf = False

max_iter_SCF = 500
eps_SCF = 1E-5
Delta_SCF = 0

The exact one-body integral function is

In [5]:
def analytical_1(p,q):
    I = 3*ALPHA[p-1]*ALPHA[q-1]*np.pi**(1.5)/(ALPHA[p-1] + ALPHA[q-1])**(5/2) - 4*np.pi/(ALPHA[p-1] + ALPHA[q-1])
    
    return I

The exact two-body integral function is

In [6]:
def analytical_2(p, r, q, s):
    I = 2*np.pi**2.5 / ((ALPHA[p-1] + ALPHA[q-1])*(ALPHA[r-1] + ALPHA[s-1])*np.sqrt((ALPHA[p-1] + ALPHA[q-1] + ALPHA[r-1] + ALPHA[s-1])))
    
    return I

The covariance function for the exponents of the gaussians is

In [7]:
def f_cov(p, r, q, s):
    cov = 0.5*np.diag([1/(ALPHA[p-1] + ALPHA[q-1])]*3 + [1/(ALPHA[r-1] + ALPHA[s-1])]*3)
    
    return cov

We then calculate and store the one- and two-body integrals using lib.py

In [8]:
# One- and Two-body integrals
MC_args = {"f_cov":f_cov, "f_integrand":He_two_body_integrand, "N_points":N_points}
integrals = lib.integral_master(N_basis)
integrals.calculate(integrals_file, analytical_1, analytical_2 = None, MC_args = MC_args)

We calculate the overlap matrix S

In [9]:
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(ALPHA[p]+ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
print('S',S)

S [[1.20975063e+01 2.91187719e+00 3.71330014e-01 2.30637530e-02]
 [2.91187719e+00 1.42134692e+00 2.99025043e-01 2.22459699e-02]
 [3.71330014e-01 2.99025043e-01 1.41564997e-01 1.89120383e-02]
 [2.30637530e-02 2.22459699e-02 1.89120383e-02 8.24921040e-03]]


Finally, we run the Self-Consistent Field for Hartree Fock

In [10]:
lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)

E = -2.1313120 | N(SCF) = 1
E = -2.3035436 | N(SCF) = 2
E = -3.0676657 | N(SCF) = 3
E = -2.8216419 | N(SCF) = 4
E = -2.8726314 | N(SCF) = 5
E = -2.8545618 | N(SCF) = 6
E = -2.8553869 | N(SCF) = 7
E = -2.8550811 | N(SCF) = 8
E = -2.8551852 | N(SCF) = 9
E = -2.8551957 | N(SCF) = 10
E = -2.8552497 | N(SCF) = 11
E = -2.8551789 | N(SCF) = 12
E = -2.8552862 | N(SCF) = 13
E = -2.8552106 | N(SCF) = 14
E = -2.8551435 | N(SCF) = 15
E = -2.8553695 | N(SCF) = 16
E = -2.8549341 | N(SCF) = 17
E = -2.8561773 | N(SCF) = 18
E = -2.8519338 | N(SCF) = 19
E = -2.8622879 | N(SCF) = 20
E = -2.8417700 | N(SCF) = 21
E = -2.8814239 | N(SCF) = 22
E = -2.8036948 | N(SCF) = 23
E = -2.9324586 | N(SCF) = 24
E = -2.5405113 | N(SCF) = 25
E = -2.9122363 | N(SCF) = 26
E = -2.8001166 | N(SCF) = 27
E = -2.9949418 | N(SCF) = 28
E = -2.7682160 | N(SCF) = 29
E = -2.7866164 | N(SCF) = 30
E = -2.7646600 | N(SCF) = 31
E = -2.7951934 | N(SCF) = 32
E = -2.8560387 | N(SCF) = 33
E = -2.9158309 | N(SCF) = 34
E = -2.8110892 | N(SCF)

E = -2.9130629 | N(SCF) = 388
E = -2.7678237 | N(SCF) = 389
E = -2.9961642 | N(SCF) = 390
E = -2.3470074 | N(SCF) = 391
E = -2.5038203 | N(SCF) = 392
E = -2.6602360 | N(SCF) = 393
E = -2.7681635 | N(SCF) = 394
E = -2.9695407 | N(SCF) = 395
E = -2.6973087 | N(SCF) = 396
E = -2.7485104 | N(SCF) = 397
E = -2.9555744 | N(SCF) = 398
E = -2.6327922 | N(SCF) = 399
E = -3.1446457 | N(SCF) = 400
E = -2.4131107 | N(SCF) = 401
E = -3.3285899 | N(SCF) = 402
E = -2.4773810 | N(SCF) = 403
E = -3.2018776 | N(SCF) = 404
E = -2.5363796 | N(SCF) = 405
E = -2.8171959 | N(SCF) = 406
E = -2.7866565 | N(SCF) = 407
E = -2.8398806 | N(SCF) = 408
E = -2.8732232 | N(SCF) = 409
E = -2.8616334 | N(SCF) = 410
E = -2.8726419 | N(SCF) = 411
E = -2.8531679 | N(SCF) = 412
E = -2.8599713 | N(SCF) = 413
E = -2.8469974 | N(SCF) = 414
E = -2.8560230 | N(SCF) = 415
E = -2.8676608 | N(SCF) = 416
E = -2.8090205 | N(SCF) = 417
E = -2.9201723 | N(SCF) = 418
E = -2.5555065 | N(SCF) = 419
E = -2.7728714 | N(SCF) = 420
E = -3.089